# Wilkommen zur diesjährigen Team-Vision Strava Auswertung 🎉
**Deadline:**
Die in Zelle 2 entstehende "2023_strava_activities.csv" bitte bis Mittwoch, **30.11.2023 (EoD) an mich (E-Mail, Messanger, ...)**.

Es werden die Rides (Virtuell und RealWorld) vom 01.01 - 30.11.2022 ausgewertet. Freut euch auf ein bisschen alljährliche Zahlen-Magic 😏!

## Anleitung:

1. In Strava anmelden
2.  www.strava.com/settings/api aufrufen
  (wenn ihr letztes Jahr schon mitgemacht habt, ist Schritt 2 & 3 nicht nochmal nötig):
  - Application Name: **Irgendwas**
  - Category: **Irgendwas**
  - Club: **Leer**
  - Website: **https://beliebig.eu** (ist komplett egal, darf aber nicht leer sein)
  - Application Description: **leer**
  - Authorization Callback Domain: **localhost**
  - Haken setzen + Create
3. Bild einfügen als Icon - was ist egal.
Sehr gut, du hast jetzt einen API Zugang (1000Request Daily, 100 every 15min)
4. Authentification:
 - Gib das hier im Browser ein:
  ```
  http://www.strava.com/oauth/authorize?client_id=[REPLACE_WITH_YOUR_CLIENT_ID]&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all
  ```
  Wichtig ist das du die [ ] duch deine *client ID* ersetzt. Die findest du in deiner neu kreierten API
  - Lass die "App" auf deinen Acoount zugreifen (bestätigen).
  - Jetzt kommst du auf eine "Non-Found" seite. Das ist genau richtig. Was wir aus dieser **URL** brauchen ist alles was hinter *code=* steht bis zu dem *&*. Das bitte kopieren
5. Jetzt geht es hier weiter:
  - Alle Infos bei "data" einfüllen (*client_id, secret, code*)
  - Dann ausführen
  - Auch im nächsten Frame XXXX befüllen



In [1]:
import requests
import json

response = requests.post(
    url = 'https://www.strava.com/oauth/token',
    data = {
        'client_id': XXXX,
        'client_secret': 'XXXXX',
        'code': 'XXXXX',
        'grant_type': 'authorization_code'
        }
)


strava_tokens = response.json()

with open ('strava_tokens.json', 'w') as outfile:
  json.dump(strava_tokens, outfile)

with open ('strava_tokens.json') as check:
  data = json.load(check)
print(data)

NameError: ignored

In [2]:
import pandas as pd
import time


with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
if strava_tokens['expires_at'] < time.time():
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': XXXXX,
                                'client_secret': 'XXXXXX',
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    )#Save response as json in new variable

    new_strava_tokens = response.json()# Save new tokens to file

    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)#Use new Strava tokens from now
        strava_tokens = new_strava_tokens#Loop through all activities

page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
r_filtered = []

while True:

    # get page of activities from Strava
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()# if no results then exit loop

    for i in r:
      if ((i['type'] == 'VirtualRide') or (i['type'] == 'Ride')) and ('2023-' in i['start_date_local']):
        r_filtered.append(i)

    if (not r):
        break

    page += 1


activities = pd.DataFrame(
    columns = [
            "id",
            "name",
            "start_date_local",
            "type",
            "visibility",
            "distance", # in Meters
            "moving_time",
            "elapsed_time",
            "total_elevation_gain",
            "average_speed", # Meters per second
            "average_cadence",
            "average_watts",
            "device_watts",
            "end_latlng",
            "kudos_count",
            "athlete_count",
            "total_photo_count",
            "external_id"
    ]
)
while float('NaN') in r_filtered:
    r_filtered.remove(float('NaN'))

for x in range(len(r_filtered)):
  try:
      activities.loc[x,'id'] = r_filtered[x]['id']
      activities.loc[x,'name'] = r_filtered[x]['name']
      activities.loc[x,'start_date_local'] = r_filtered[x]['start_date_local']
      activities.loc[x,'type'] = r_filtered[x]['type']
      activities.loc[x,'visibility'] = r_filtered[x]['visibility']
      activities.loc[x,'distance'] = r_filtered[x]['distance']
      activities.loc[x,'moving_time'] = r_filtered[x]['moving_time']
      activities.loc[x,'elapsed_time'] = r_filtered[x]['elapsed_time']
      activities.loc[x,'total_elevation_gain'] = r_filtered[x]['total_elevation_gain']
      activities.loc[x,'average_speed'] = r_filtered[x]['average_speed']
      activities.loc[x,'average_cadence'] = r_filtered[x]['average_cadence']
      activities.loc[x,'average_watts'] = r_filtered[x]['average_watts']
      activities.loc[x,'device_watts'] = r_filtered[x]['device_watts']
      activities.loc[x,'end_latlng'] = r_filtered[x]['end_latlng']
      activities.loc[x,'kudos_count'] = r_filtered[x]['kudos_count']
      activities.loc[x,'athlete_count'] = r_filtered[x]['athlete_count']
      activities.loc[x,'total_photo_count'] = r_filtered[x]['total_photo_count']
      activities.loc[x,'external_id'] = r_filtered[x]['external_id']
  except KeyError: pass


activities.to_csv('2023_strava_bike-activities.csv')
print("Done - File sollte in diesem Verzeichnis liegen. Bitte an Bene schicken.")

FileNotFoundError: ignored